In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import time
import json
import pprint
import random

In [2]:
# api Key
from key import gkey

## Retrieve data from API

In [3]:
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

#this keyword will be replaced by the names of the desirable dishes


# This coordinate is for the city of Los Angeles
coordinates = "34.0522, -118.2437"
radius = 10000
search_type = 'restaurant'



In [4]:
random.randint(0, 20)

13

## Transfrom: Import the recipe data from the cvs file and clean up data

In [5]:
path = "Resources/Recipes-All Recipes.csv"

In [6]:
recipes_df = pd.read_csv(path)

In [7]:
recipes_df

,Name,Rating,Ease of Prep,Notes,Type,Prep Time,Photo,Cookbook,Page #,Ingredients,Slowcooker,Link,Last Made,Make It Next
0,Jamaican Jerk Chicken,⭐⭐⭐⭐⭐,Average,Luke loved this one,Main Dish,30.0,NaN,Taste From Home,NaN,Chicken,NaN,NaN,12/8/2018,12/28/2018
1,Pasta Salad,⭐⭐⭐,Fairly Easy,Make night before,Salad,20.0,NaN,Taste From Home,NaN,"Kielbasa,Noodles",NaN,NaN,NaN,#ERROR
2,Lasagna,⭐⭐⭐,Fairly Easy,Make night before,Main Dish,45.0,NaN,Taste From Home,NaN,"Sausage,Noodles",NaN,NaN,1/6/2019,2/22/2019
3,Hearty Pancakes,⭐⭐⭐⭐,Average,Makes enough for 2 days,Breakfast,20.0,Image 29 April 2017 07:50 AM.jpg (https://dl.a...,Taste From Home,NaN,"Milk,Eggs,Corn Meal",NaN,NaN,NaN,#ERROR
4,Summer Garden Couscous Salad,⭐⭐⭐⭐,Hard,NaN,Salad,30.0,Image 08 May 2017 07:00 PM.jpg (https://dl.air...,Taste of Home,287.0,Vegetarian,NaN,NaN,11/18/2018,12/18/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,Roasted Mackerel,NaN,NaN,NaN,NaN,NaN,NaN,Taste of Home,NaN,NaN,NaN,NaN,11/30/2018,11/30/2018
141,Lentil Salsa Soup,⭐⭐⭐⭐,NaN,NaN,NaN,NaN,NaN,Taste of Home,NaN,NaN,NaN,NaN,12/7/2018,1/6/2019
142,Pesto Tomato Penne,⭐⭐⭐,Super Simple,NaN,Main Dish,20.0,NaN,NaN,NaN,Pesto,NaN,https://www.foodnetwork.com/recipes/giada-de-l...,1/13/2019,3/1/2019
143,Black Bean Soup,⭐⭐⭐⭐,Fairly Easy,NaN,Main Dish,30.0,NaN,Taste From Home,NaN,NaN,NaN,NaN,2/13/2019,3/15/2019


In [8]:
# Clean up the dataframe,
# Drop all the irrelevant columns from the original csv file
recipes_df.drop(['Notes', 'Photo', 'Cookbook', 'Page #', 'Slowcooker', 'Link', 'Last Made', 'Make It Next'], axis=1)

,Name,Rating,Ease of Prep,Type,Prep Time,Ingredients
0,Jamaican Jerk Chicken,⭐⭐⭐⭐⭐,Average,Main Dish,30.0,Chicken
1,Pasta Salad,⭐⭐⭐,Fairly Easy,Salad,20.0,"Kielbasa,Noodles"
2,Lasagna,⭐⭐⭐,Fairly Easy,Main Dish,45.0,"Sausage,Noodles"
3,Hearty Pancakes,⭐⭐⭐⭐,Average,Breakfast,20.0,"Milk,Eggs,Corn Meal"
4,Summer Garden Couscous Salad,⭐⭐⭐⭐,Hard,Salad,30.0,Vegetarian
...,...,...,...,...,...,...
140,Roasted Mackerel,NaN,NaN,NaN,NaN,NaN
141,Lentil Salsa Soup,⭐⭐⭐⭐,NaN,NaN,NaN,NaN
142,Pesto Tomato Penne,⭐⭐⭐,Super Simple,Main Dish,20.0,Pesto
143,Black Bean Soup,⭐⭐⭐⭐,Fairly Easy,Main Dish,30.0,NaN


In [9]:
len(recipes_df.Name)

145

In [10]:
recipes_df['Name'] = recipes_df["Name"].astype(str)

# Extracting: getting name and the address of the restaurant

In [11]:
keyword = recipes_df.Name[0]

In [12]:
# response = requests.get(base_url, params=param)
# place_data = response.json()

In [13]:
# print(place_data['results'][0]['name'])
# print(place_data['results'][0]['formatted_address'])

In [14]:
list_of_food_and_restaurant= []

#base URL for Google API
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# basic parameters: city of LA's lat and long, radius in 10000 meters, and looking for restaurants
coordinates = "34.0522, -118.2437"
radius = 10000
search_type = 'restaurant'


# set up a counter to check when the api have run completely
counter = 0

In [15]:
for dish in recipes_df.Name:
    # set the keyword as the name of the dish
    keyword = dish
    params = {'keyword': keyword, 
         'location': coordinates, 
         'radius': radius, 
         "type": search_type,
         "key" : gkey}
    
    # send requests to retrieve the data
    response = requests.get(base_url, params=params)
    # save the response (as json format) to Place_data variable
    place_data = response.json()
    
    
    # Set delay timer for 1 sec, slow down the api request to avoid errors
    time.sleep(1)
    
    # update the counter:
    counter += 1
    
    try:
        # make it more fun:
        # instead of picking the first 3 restaurants, pick the first one. And let the other 2 be random
        random_rest_2 = random.randint(0, 9)
        random_rest_3 = random.randint(10, 20)
         
        # retrieve the restaurants' names
        restaurant_name1 = place_data['results'][0]['name']
        restaurant_name2 = place_data['results'][random_rest_2]['name']
        restaurant_name3 = place_data['results'][random_rest_3]['name']
        
        #retieve the addresses
        restaurant_address1 = place_data['results'][0]['formatted_address']
        restaurant_address2 = place_data['results'][random_rest_2]['formatted_address']
        restaurant_address3 = place_data['results'][random_rest_3]['formatted_address']
        
        # print out the entries to keep track of the process
        print('-----------------------------')
        print(dish)
        print(restaurant_name1,',', restaurant_name2,',', restaurant_name3)
        print(restaurant_address1, '\n', restaurant_address2, '\n', restaurant_address3)
        print('------------------------------')
        list_of_food_and_restaurant.append({'dish_name' : dish,
                                           'restaurant_name': [restaurant_name1, 
                                                               restaurant_name2, 
                                                               restaurant_name3],
                                           'restaurant_address': [restaurant_address1, 
                                                                  restaurant_address2,
                                                                 restaurant_address3]})
    except:
        print('Error: something wrong')
        
    if counter == len(recipes_df.Name):
        print('**************************')
        print('Process is completed')

-----------------------------
Jamaican Jerk Chicken
Home Restaurant , Happy Tom's Restaurant , Rossoblu
1760 Hillhurst Ave, Los Angeles, CA 90027 
 1807 Sunset Blvd, Los Angeles, CA 90026 
 CITY MARKET SOUTH, 1124 San Julian St, Los Angeles, CA 90015
------------------------------
-----------------------------
Pasta Salad
Home Restaurant , Yard House , Marino Restaurant
1760 Hillhurst Ave, Los Angeles, CA 90027 
 800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 6001 Melrose Ave, Los Angeles, CA 90038
------------------------------
-----------------------------
Lasagna
Home Restaurant , Home Restaurant , Bottega Louie
1760 Hillhurst Ave, Los Angeles, CA 90027 
 1760 Hillhurst Ave, Los Angeles, CA 90027 
 700 S Grand Ave, Los Angeles, CA 90017
------------------------------
-----------------------------
Hearty Pancakes
Home Restaurant , Redbird , Five Guys
1760 Hillhurst Ave, Los Angeles, CA 90027 
 114 E 2nd St, Los Angeles, CA 90012 
 530 W 27th St Suite 101, Los Angeles, CA 90007
---

-----------------------------
S'mores Cookie Bars
Home Restaurant , Bestia , Marino Restaurant
1760 Hillhurst Ave, Los Angeles, CA 90027 
 2121 E 7th Pl, Los Angeles, CA 90021 
 6001 Melrose Ave, Los Angeles, CA 90038
------------------------------
-----------------------------
Orange Chicken
Home Restaurant , Home Restaurant , Perch
1760 Hillhurst Ave, Los Angeles, CA 90027 
 1760 Hillhurst Ave, Los Angeles, CA 90027 
 448 S Hill St, Los Angeles, CA 90013
------------------------------
-----------------------------
Tostadas
Home Restaurant , 71Above , Bottega Louie
1760 Hillhurst Ave, Los Angeles, CA 90027 
 633 W 5th St 71st floor, Los Angeles, CA 90071 
 700 S Grand Ave, Los Angeles, CA 90017
------------------------------
-----------------------------
Black Bean Stuffed Sweet Potatoes
Home Restaurant , Guisados , Rossoblu
1760 Hillhurst Ave, Los Angeles, CA 90027 
 1261 Sunset Blvd, Los Angeles, CA 90026 
 CITY MARKET SOUTH, 1124 San Julian St, Los Angeles, CA 90015
---------------

-----------------------------
Italian Vegetable Lentil Soup
Yard House , Perch , Chichen Itza
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 448 S Hill St, Los Angeles, CA 90013 
 3655 S Grand Ave c6, Los Angeles, CA 90007
------------------------------
-----------------------------
One Pot Chicken & Potatoes
Yard House , Bar Restaurant , Kismet
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 4326 Sunset Blvd, Los Angeles, CA 90029 
 4648 Hollywood Blvd, Los Angeles, CA 90027
------------------------------
-----------------------------
Sweet Korean Lentils
Yard House , APL Restaurant , Marino Restaurant
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 1680 Vine St, Los Angeles, CA 90028 
 6001 Melrose Ave, Los Angeles, CA 90038
------------------------------
-----------------------------
Buckwheat Beetroot Salad
Yard House , Fleming’s Prime Steakhouse & Wine Bar , Marino Restaurant
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 800 W Olympic Blvd Suite A-135, Los Angeles, CA 9

-----------------------------
Spanish Chickpeas
Yard House , Fleming’s Prime Steakhouse & Wine Bar , Zinc Cafe & Market and Bar
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 800 W Olympic Blvd Suite A-135, Los Angeles, CA 90015 
 580 Mateo St, Los Angeles, CA 90013
------------------------------
Error: something wrong
-----------------------------
Chickpea Masala
Yard House , Redbird , Marino Restaurant
800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 114 E 2nd St, Los Angeles, CA 90012 
 6001 Melrose Ave, Los Angeles, CA 90038
------------------------------
-----------------------------
Chickpea Broccoli Pesto
Home Restaurant , Chichen Itza , Rossoblu
1760 Hillhurst Ave, Los Angeles, CA 90027 
 3655 S Grand Ave c6, Los Angeles, CA 90007 
 CITY MARKET SOUTH, 1124 San Julian St, Los Angeles, CA 90015
------------------------------
Error: something wrong
Error: something wrong
Error: something wrong
-----------------------------
Balsamic Potatoes and Asparagus
Home Restaurant , Home

-----------------------------
Sweet Potato Lentil Curry with Pickled Onion
Home Restaurant , Chichen Itza , NORMS Restaurant
1760 Hillhurst Ave, Los Angeles, CA 90027 
 3655 S Grand Ave c6, Los Angeles, CA 90007 
 2500 E, 2500 Slauson Ave, Huntington Park, CA 90255
------------------------------
-----------------------------
Sardine Mediterranean Pasta
Home Restaurant , Redbird , Bricks Restaurant & Sports Bar
1760 Hillhurst Ave, Los Angeles, CA 90027 
 114 E 2nd St, Los Angeles, CA 90012 
 3626 Fruitland Ave, Maywood, CA 90270
------------------------------
-----------------------------
Prosciutto apple flatbread pizza
Home Restaurant , Yard House , Rossoblu
1760 Hillhurst Ave, Los Angeles, CA 90027 
 800 W Olympic Blvd A-115, Los Angeles, CA 90015 
 CITY MARKET SOUTH, 1124 San Julian St, Los Angeles, CA 90015
------------------------------
-----------------------------
Dill Cucumber Salmon
Home Restaurant , CAVA , Five Guys
1760 Hillhurst Ave, Los Angeles, CA 90027 
 3201 S Hoover St

## Transform:
* Transfrom process has been done in 2 part:
    * First transform for the csv file: cleaning out unnessary columns
    * Second transform has been performed during the appending into the list: filtering out only the names and the address of the restaurants, not the whole json from the APIs

## Load: get the list into MongoDB (noSQL)

In [16]:
# list of the data is in the correct format (dictinary/JSON)
list_of_food_and_restaurant

[{'dish_name': 'Jamaican Jerk Chicken',
  'restaurant_name': ['Home Restaurant', "Happy Tom's Restaurant", 'Rossoblu'],
  'restaurant_address': ['1760 Hillhurst Ave, Los Angeles, CA 90027',
   '1807 Sunset Blvd, Los Angeles, CA 90026',
   'CITY MARKET SOUTH, 1124 San Julian St, Los Angeles, CA 90015']},
 {'dish_name': 'Pasta Salad',
  'restaurant_name': ['Home Restaurant', 'Yard House', 'Marino Restaurant'],
  'restaurant_address': ['1760 Hillhurst Ave, Los Angeles, CA 90027',
   '800 W Olympic Blvd A-115, Los Angeles, CA 90015',
   '6001 Melrose Ave, Los Angeles, CA 90038']},
 {'dish_name': 'Lasagna',
  'restaurant_name': ['Home Restaurant', 'Home Restaurant', 'Bottega Louie'],
  'restaurant_address': ['1760 Hillhurst Ave, Los Angeles, CA 90027',
   '1760 Hillhurst Ave, Los Angeles, CA 90027',
   '700 S Grand Ave, Los Angeles, CA 90017']},
 {'dish_name': 'Hearty Pancakes',
  'restaurant_name': ['Home Restaurant', 'Redbird', 'Five Guys'],
  'restaurant_address': ['1760 Hillhurst Ave, L

In [17]:
import pymongo

In [18]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [19]:
# Create a new mongo database with name as recipe_at_restaurant_DB
db = client.recipe_at_restaurant_DB

In [20]:
# connect (or create if not yet exist) to collection of recipe_restaurant
recipe_restaurant = db.recipe_restaurant.find()

In [21]:
recipe_restaurant

In [22]:
# start insert the whole list of retrieved data into the database
db.recipe_restaurant.insert_many(list_of_food_and_restaurant)

In [23]:
#querry the recipe_restaurant collection
recipe_restaurant = db.recipe_restaurant.find()

In [24]:
for item in recipe_restaurant:
    print(item)

{'_id': ObjectId('61b5215c768deb1f203b1d4d'), 'dish_name': 'Jamaican Jerk Chicken', 'restaurant_name': ['JOEY DTLA', 'Water Grill', 'NORMS Restaurant'], 'restaurant_address': ['700 W 7th St Ste S430, Los Angeles, CA 90017', '544 S Grand Ave, Los Angeles, CA 90071', '2500 E, 2500 Slauson Ave, Huntington Park, CA 90255']}
{'_id': ObjectId('61b5215c768deb1f203b1d4e'), 'dish_name': 'Pasta Salad', 'restaurant_name': ['JOEY DTLA', 'Teresitas Restaurant', 'Spitz - Little Tokyo Restaurant & Bar'], 'restaurant_address': ['700 W 7th St Ste S430, Los Angeles, CA 90017', '3826 E 1st St, Los Angeles, CA 90063', '371 E 2nd St, Los Angeles, CA 90012']}
{'_id': ObjectId('61b5215c768deb1f203b1d4f'), 'dish_name': 'Lasagna', 'restaurant_name': ['JOEY DTLA', 'Chichen Itza', 'Fogo de Chão Brazilian Steakhouse'], 'restaurant_address': ['700 W 7th St Ste S430, Los Angeles, CA 90017', '3655 S Grand Ave c6, Los Angeles, CA 90007', '800 S Figueroa St, Los Angeles, CA 90017']}
{'_id': ObjectId('61b5215c768deb1f2

### Reason we think noSQL is better for this kind of data:
* there info are just string type about names and addresses. There is no need to conduct mathmatical calculations
* we can add on more entries in the future if we can find more recipes
* We don't have to modify too much when we change/update the recipes